In [16]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("./input"))

import lightgbm as lgb
from bayes_opt import BayesianOptimization

from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

['.DS_Store', 'feature_report.csv', 'sample_submission.csv', 'test.csv', 'test_leak_org.csv', 'train.csv', 'train_leak_org.csv']


In [3]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]

In [4]:
train_leak = pd.read_csv("./input/train_leak_org.csv")
test_leak = pd.read_csv("./input/test_leak_org.csv")
train_leak = train_leak.replace(np.nan,0.0)
test_leak = test_leak.replace(np.nan,0.0)

In [5]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [6]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]

In [27]:
best_lag = 29
train_leak = rewrite_compiled_leak(train_leak, best_lag)
test_leak = rewrite_compiled_leak(test_leak, best_lag)
train_leak.head(5)

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000d6aaf2,38000000.0,38000000.0,38000000.0,0.0,38000000.0,0.0,38000000.0,0.0,0.0,...,38000000.0,0.0,38000000.0,0.0,38000000.0,38000000.0,0.0,0.0,0.0,38000000.0
1,000fbd867,600000.0,0.0,0.0,0.0,0.0,0.0,600000.0,0.0,600000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0
2,0027d6b71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0028cbf45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0
4,002a68644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
new_train = test.loc[test_leak["compiled_leak"] > 0].copy()
new_train["target"] = test_leak["compiled_leak"].loc[test_leak["compiled_leak"] > 0]
new_train["leak"] = new_train["target"]
new_train['log_leak'] = np.log1p(new_train["leak"])

_temp_train = train.copy()
_temp_train["leak"] = train_leak['compiled_leak']
_temp_train['log_leak'] = np.log1p(_temp_train["leak"])

new_train = pd.concat([_temp_train, new_train]).reset_index(drop=True)
new_test = test.loc[test_leak["compiled_leak"] == 0].copy().reset_index(drop=True)
new_test['leak'] = 0
new_test['log_leak'] = 0

In [29]:
report = pd.read_csv("./input/feature_report.csv")
good_features = report.loc[report['rmse'] <= 0.7925]["feature"].values
rmses = report.loc[report['rmse'] <= 0.7925, 'rmse'].values

In [30]:
target = np.log1p(new_train['target'])

folds = KFold(n_splits=5, shuffle=True, random_state=1)

features = [f for f in new_train if f not in ['ID', 'leak', 'log_leak', 'target']]

new_train.replace(0, np.nan, inplace=True)
new_train['log_of_mean'] = np.log1p(new_train[features].replace(0, np.nan).mean(axis=1))
new_train['mean_of_log'] = np.log1p(new_train[features]).replace(0, np.nan).mean(axis=1)
new_train['log_of_median'] = np.log1p(new_train[features][features].replace(0, np.nan).median(axis=1))
new_train['nb_nans'] = new_train[features].isnull().sum(axis=1)
new_train['the_sum'] = np.log1p(new_train[features].sum(axis=1))
new_train['the_std'] = new_train[features].std(axis=1)
new_train['the_kur'] = new_train[features].kurtosis(axis=1)

new_test.replace(0, np.nan, inplace=True)
new_test['log_of_mean'] = np.log1p(new_test[features].replace(0, np.nan).mean(axis=1))
new_test['mean_of_log'] = np.log1p(new_test[features]).replace(0, np.nan).mean(axis=1)
new_test['log_of_median'] = np.log1p(new_test[features].replace(0, np.nan).median(axis=1))
new_test['nb_nans'] = new_test[features].isnull().sum(axis=1)
new_test['the_sum'] = np.log1p(new_test[features].sum(axis=1))
new_test['the_std'] = new_test[features].std(axis=1)
new_test['the_kur'] = new_test[features].kurtosis(axis=1)

features = good_features.tolist()
features = features + ['log_leak', 'log_of_mean', 'mean_of_log', 'log_of_median', 'nb_nans', 'the_sum', 'the_std', 'the_kur']

In [17]:
def lgb_evaluate(num_leaves, subsample, colsample_bytree,
                min_split_gain, reg_alpha, reg_lambda,
                min_child_weight, verbose, learning_rate):
    oof_preds = np.zeros(new_train.shape[0])
    for trn_idx, val_idx in folds.split(new_train):
        if int(verbose) != 0:
            verbose = int(verbose)
        else:
            verbose = 1
            
        lgb_params = {
            'objective': 'regression',
            'num_leaves': int(num_leaves),
            'subsample': subsample,
            'colsample_bytree': colsample_bytree,
            'min_split_gain': min_split_gain,
            'reg_alpha': reg_alpha,
            'reg_lambda': reg_lambda,
            'min_child_weight': min_child_weight,
            'verbose': verbose,
            'seed': 3,
            'boosting_type': 'gbdt',
            'max_depth': -1,
            'learning_rate': learning_rate,
            'metric': 'l2',
        }
        
        clf = lgb.train(
            params=lgb_params,
            train_set=dtrain.subset(trn_idx),
            valid_sets=dtrain.subset(val_idx),
            num_boost_round=10000, 
            early_stopping_rounds=100,
            verbose_eval=0
        )
        oof_preds[val_idx] = clf.predict(dtrain.data.iloc[val_idx])s
    result = mean_squared_error(target[:len(train)], oof_preds[:len(train)]) ** .5
    return -result


In [23]:
dtrain = lgb.Dataset(data=new_train[features], 
                     label=target, free_raw_data=False)
new_test['target'] = 0
dtrain.construct()
num_iter = 25
init_points = 5

lgbBO = BayesianOptimization(lgb_evaluate, {'num_leaves': (30, 100),
                                            'subsample': (0.01,1),
                                            'colsample_bytree': (0.01, 1),
                                            'min_split_gain': (0.0001, 10),
                                            'reg_alpha': (0, 10),
                                            'reg_lambda': (0, 20),
                                            'min_child_weight': (0,10),
                                            'verbose': (-10, 10),
                                            'learning_rate': (0.00001, 10),
                                           })
lgbBO.initialize({"target": [-0.648801],
               'num_leaves': [58],
               'subsample': [0.6143],
               'colsample_bytree': [0.6453],
               'min_split_gain': [np.power(10, -2.5988)],
               'reg_alpha': [np.power(10, -2.2887)],
               'reg_lambda': [np.power(10, 1.7570)],
               'min_child_weight': [np.power(10, -0.1477)],
               'verbose': [-1],
               'learning_rate': [0.05],
              })
lgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   learning_rate |   min_child_weight |   min_split_gain |   num_leaves |   reg_alpha |   reg_lambda |   subsample |   verbose | 
    1 | 00m13s |   -0.79477 |             0.0347 |          1.2944 |             9.4105 |           8.4027 |       2.4449 |      2.6235 |      14.8738 |      0.4387 |   -7.4053 | 
    2 | 00m01s |   -3.21399 |             0.2433 |          3.0163 |             6.2000 |           7.9447 |      63.7919 |      8.3802 |      18.4896 |      0.2443 |    7.0786 | 
    3 | 00m01s |   -3.17004 |             0.0434 |          3.6635 |             7.7144 |           5.7870 |      28.5865 |      8.3374 |      18.2938 |      0.0135 |    5.8665 | 
    4 | 00m01s |   -2.79681 |             0.0891 |          3.2123 |             0.035

In [12]:
new_test['target'] = np.expm1(new_test['target'])
new_test.head(11)

,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,leak,log_leak,log_of_mean,mean_of_log,log_of_median,nb_nans,the_sum,the_std,the_kur,target
0,000137c73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,17.159699,14.608343,15.428193,4913,21.516408,1.121562e+08,63.987338,1.961166e+06
1,00021489f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,15.854051,13.910498,13.983117,4979,18.338957,1.021367e+07,1.322530,1.402277e+06
2,0004d7953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,15.595720,14.059439,14.508658,4892,20.190840,1.152589e+07,19.210890,1.559789e+06
3,00056a333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16.507243,15.878736,16.198495,4871,21.294735,1.398462e+07,10.568926,3.682574e+06
4,00056d8eb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16.626928,14.869434,15.036607,4983,18.706369,3.691364e+07,7.851651,9.996476e+05
5,0005fc190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16.238255,15.286862,15.753069,4969,19.329298,1.352289e+07,0.916292,2.617852e+06
6,000787e86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16.688396,13.885650,14.607098,4966,19.907272,3.024605e+07,4.019249,1.756854e+06
7,0008510a0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16.302419,15.266272,15.332173,4984,18.248329,1.663716e+07,1.843609,1.762901e+06
8,000895faf,NaN,NaN,2.461712e+07,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16.257260,14.830404,15.456254,4963,19.589464,1.589470e+07,4.704776,3.242100e+06
9,000986fba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12675000.0,...,NaN,NaN,15.701677,14.765847,15.053885,4843,20.698889,1.019439e+07,20.910049,1.567430e+06


In [ ]:
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.loc[sub["target"] > 0, "target"] = np.expm1(oof_preds[len(train):])
sub.loc[sub["target"] == 0, "target"] = new_test['target'].values
#sub.head(12)

In [ ]:
#sub = test[["ID"]]
#sub["target"] = test_leak["compiled_leak"]
#sub.loc[sub["target"] == 0, "target"] = new_test['target'].values
#sub.head(12)

In [ ]:
sub.to_csv(f"lgb_and_leak_{best_lag}.csv", index=False)
print(f"lgb_and_leak_{best_lag}.csv saved")

In [ ]:
#train_res = train_leak[["ID"]+leaky_cols+['compiled_leak']].replace(0.0, np.nan)
#train_res.to_csv('train_leak.csv', index=False)
#print(f"train_leak.csv saved")

In [ ]:
#xtest_res = test_leak[["ID"]+leaky_cols+['compiled_leak']].replace(0.0, np.nan)
#xtest_res.to_csv('test_leak.csv', index=False)
#xprint(f"test_leak.csv saved")

In [31]:
#dtrain = lgb.Dataset(data=new_train[features], 
#                     label=target, free_raw_data=False)
#new_test['target'] = 0
#dtrain.construct()
#
#oof_preds = np.zeros(new_train.shape[0])
#
#for trn_idx, val_idx in folds.split(new_train):
#    lgb_params = {
#        'objective': 'regression',
#        'num_leaves': 58,
#        'subsample': 0.6143,
#        'colsample_bytree': 0.6453,
#        'min_split_gain': np.power(10, -2.5988),
#        'reg_alpha': np.power(10, -2.2887),
#        'reg_lambda': np.power(10, 1.7570),
#        'min_child_weight': np.power(10, -0.1477),
#        'verbose': -1,
#        'seed': 3,
#        'boosting_type': 'gbdt',
#        'max_depth': -1,
#        'learning_rate': 0.05,
#        'metric': 'l2',
#    }
#
#    clf = lgb.train(
#        params=lgb_params,
#        train_set=dtrain.subset(trn_idx),
#        valid_sets=dtrain.subset(val_idx),
#        num_boost_round=10000, 
#        early_stopping_rounds=100,
#        verbose_eval=0
#    )
#
#    oof_preds[val_idx] = clf.predict(dtrain.data.iloc[val_idx])
#    new_test['target'] += clf.predict(new_test[features]) / folds.n_splits
#    #print(mean_squared_error(target.iloc[val_idx], 
#    #                         oof_preds[val_idx]) ** .5)
#
#new_train['predictions'] = oof_preds
#new_train.loc[new_train['leak'].notnull(), 'predictions'] = np.log1p(new_train.loc[new_train['leak'].notnull(), 'leak'])
#print('OOF SCORE : %9.6f' 
#      % (mean_squared_error(target[:len(train)], oof_preds[:len(train)]) ** .5)) #[:len(train)]
#print('OOF SCORE with LEAK : %9.6f' 
#      % (mean_squared_error(target[:len(train)], new_train['predictions'][:len(train)]) ** .5))




OOF SCORE :  0.648801
OOF SCORE with LEAK :  0.649912
